In [161]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from time import time

np.random.seed(1337)
df = pd.read_csv('train.csv') 
df = df.drop(['Name','Ticket', 'Cabin'], axis=1)

In [162]:
df = df.dropna() # drop rows with missing ages.

In [163]:
# To convert categorical data to numerical data, a few different ways.

# 1.
# df['Sex'] = df['Sex'].map({'female':0, 'male':1}) # change sex to integer values

# 2.
# embarked_sex = pd.get_dummies(df.Sex, prefix="Sex").iloc[:,1:] # change Sex to Sex_male
# embarked_dummies = pd.get_dummies(df.Embarked, prefix="Embarked").iloc[:, 1:] # Embarked_S, Embarked_Q

# concat back to original data frame.
# pd = pd.concat([df, embarked_sex], axis = 1)
# pd = pd.concat([df, embarked_dummies], axis = 1)

# 3.
# A better way:
df = pd.get_dummies(df, columns = ['Sex', 'Embarked'])

In [164]:
# randomly select 80% rows for training and 20% rows for testing.
msk = np.random.rand(len(df)) < 0.8
train = df[msk]
test = df[~msk]

In [165]:
# Normalize features to the same scale.
scaler = StandardScaler()
features = ['Pclass','Sex_female','Sex_male','Age','SibSp','Parch','Fare','Embarked_C','Embarked_Q','Embarked_S']

X_train = scaler.fit_transform(train[features].values)
Y_train = scaler.fit_transform(train['Survived'].values)
y_train_onehot = pd.get_dummies(train['Survived']).values

X_test = scaler.transform(test[features].values)
y_test = test['Survived'].values


/Users/lingxiwu/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)
/Users/lingxiwu/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/lingxiwu/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


In [166]:
from keras.models import Sequential
from keras.layers import Dense, Activation

start = time()

model = Sequential()
model.add(Dense(input_dim=4, output_dim=100))
model.add(Dense(output_dim=100))
model.add(Dense(output_dim=2))
model.add(Activation("softmax"))

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

model.fit(X_train, y_train_onehot)

# print('\ntime taken %s seconds' % str(time() - start))

y_prediction = model.predict_classes(X_test)
print("\n\naccuracy", np.sum(y_prediction == y_test) / float(len(y_test)))

/Users/lingxiwu/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=4, units=100)`
  import sys
/Users/lingxiwu/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=100)`
  
/Users/lingxiwu/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=2)`
  if __name__ == '__main__':


ValueError: Error when checking input: expected dense_7_input to have shape (None, 4) but got array with shape (582, 10)